# 🧹 Data Cleaning Exercise (Single Notebook)

This notebook **generates a messy CSV dataset** and then **cleans it** step by step using **pandas** and **NumPy**.

It saves:
- `data/messy_rainfall.csv` (raw, messy data)
- `results/cleaned_rainfall.csv` (clean output)
- `results/summary.txt` (basic stats)
- `results/quick_plot.png` (sanity chart)


In [10]:
# === Imports & Setup ===
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

# Use the folder where THIS notebook lives as project root
PROJECT_ROOT = Path().resolve().parent
DATA_DIR = PROJECT_ROOT / "data"
RESULTS_DIR = PROJECT_ROOT / "results"
DATA_DIR.mkdir(exist_ok=True)
RESULTS_DIR.mkdir(exist_ok=True)

print("Project root:", PROJECT_ROOT)
print("Data dir:", DATA_DIR)
print("Results dir:", RESULTS_DIR)

Project root: D:\DS Projects\pandas-data-cleaning-project
Data dir: D:\DS Projects\pandas-data-cleaning-project\data
Results dir: D:\DS Projects\pandas-data-cleaning-project\results


## 1) Generate a messy CSV

We intentionally create common issues:
- Mixed date formats
- Numeric values as text (spaces/commas)
- Missing values
- Duplicates
- A junk row


In [11]:
# === Generate messy data ===
rows = [
    {"Date": "2024-01-01", "Rainfall(mm)": " 12.5 ", "Region": "North", "Note": "ok"},
    {"Date": "01/02/2024", "Rainfall(mm)": "NaN", "Region": "south", "Note": "error"},
    {"Date": "2024/01/03", "Rainfall(mm)": "15.0", "Region": "EAST", "Note": "ok"},
    {"Date": "", "Rainfall(mm)": "", "Region": "West", "Note": "missing"},
    {"Date": "2024-01-05", "Rainfall(mm)": "20,3", "Region": "north", "Note": "ok"},  # comma decimal
    {"Date": "junk_row", "Rainfall(mm)": "junk_value", "Region": "??", "Note": "not needed"},
    {"Date": "2024-01-05", "Rainfall(mm)": "20,3", "Region": "north", "Note": "ok"},  # duplicate
]
raw_df = pd.DataFrame(rows)

raw_path = DATA_DIR / "messy_rainfall.csv"
raw_df.to_csv(raw_path, index=False)
print(f"✅ Wrote messy dataset to: {raw_path}")
raw_df

✅ Wrote messy dataset to: D:\DS Projects\pandas-data-cleaning-project\data\messy_rainfall.csv


,Date,Rainfall(mm),Region,Note
0,2024-01-01,12.5,North,ok
1,01/02/2024,NaN,south,error
2,2024/01/03,15.0,EAST,ok
3,,,West,missing
4,2024-01-05,"20,3",north,ok
5,junk_row,junk_value,??,not needed
6,2024-01-05,"20,3",north,ok


## 2) Load & Quick Diagnostics

We read the CSV from `data/` and check the current working directory, files present, and head.


In [12]:
# === Load & diagnose ===
print("CWD:", Path().resolve())
print("Files in data/:", list(DATA_DIR.glob("*")))

df = pd.read_csv(DATA_DIR / "messy_rainfall.csv")
print("Shape:", df.shape)
df.head()

CWD: D:\DS Projects\pandas-data-cleaning-project\notebooks
Files in data/: [WindowsPath('D:/DS Projects/pandas-data-cleaning-project/data/messy_rainfall.csv')]
Shape: (7, 4)


,Date,Rainfall(mm),Region,Note
0,2024-01-01,12.5,North,ok
1,01/02/2024,NaN,south,error
2,2024/01/03,15.0,EAST,ok
3,NaN,NaN,West,missing
4,2024-01-05,"20,3",north,ok


## 3) Clean text columns

- Trim whitespace
- Standardize region casing


In [13]:
# === Clean text columns ===
obj_cols = df.select_dtypes(include="object").columns
for c in obj_cols:
    df[c] = df[c].astype(str).str.strip()

# Title-case for Region (north -> North)
if "Region" in df.columns:
    df["Region"] = df["Region"].str.title()

df.head()

,Date,Rainfall(mm),Region,Note
0,2024-01-01,12.5,North,ok
1,01/02/2024,nan,South,error
2,2024/01/03,15.0,East,ok
3,nan,nan,West,missing
4,2024-01-05,"20,3",North,ok


## 4) Parse dates safely

- Convert the `Date` column using `errors='coerce'` (invalid dates → NaT)
- Drop rows with invalid dates


In [14]:
# === Parse & validate dates ===
df["Date_parsed"] = pd.to_datetime(df["Date"], errors="coerce")
before = len(df)
df = df[df["Date_parsed"].notna()].copy()
after = len(df)
print(f"Dropped {before - after} rows with invalid dates.")
df[["Date","Date_parsed"]].head()

Dropped 4 rows with invalid dates.


,Date,Date_parsed
0,2024-01-01,2024-01-01
4,2024-01-05,2024-01-05
6,2024-01-05,2024-01-05


## 5) Normalize numeric values

- Replace comma decimals with dots (`20,3` → `20.3`)
- Strip spaces
- Convert to numeric
- Impute missing with **median**


In [15]:
# === Normalize Rainfall(mm) ===
df["Rainfall_norm"] = (
    df["Rainfall(mm)"]
    .astype(str)
    .str.replace(",", ".", regex=False)
    .str.replace(" ", "", regex=False)
)
df["Rainfall_norm"] = pd.to_numeric(df["Rainfall_norm"], errors="coerce")
median_rain = df["Rainfall_norm"].median(skipna=True)
df["Rainfall_norm"] = df["Rainfall_norm"].fillna(median_rain)

df[["Rainfall(mm)", "Rainfall_norm"]].head()

,Rainfall(mm),Rainfall_norm
0,12.5,12.5
4,"20,3",20.3
6,"20,3",20.3


## 6) Remove duplicates & keep relevant columns

We deduplicate by `Date_parsed + Region`, keep tidy columns, and sort rows.


In [16]:
# === De-duplicate & tidy columns ===
df = df.drop_duplicates(subset=["Date_parsed", "Region"])

clean = df.loc[:, ["Date_parsed", "Region", "Rainfall_norm"]].rename(
    columns={"Date_parsed": "Date", "Rainfall_norm": "Rainfall(mm)"}
)
clean = clean.sort_values(["Date", "Region"]).reset_index(drop=True)
clean.head()

,Date,Region,Rainfall(mm)
0,2024-01-01,North,12.5
1,2024-01-05,North,20.3


## 7) Save cleaned outputs

- `results/cleaned_rainfall.csv`
- `results/summary.txt`
- `results/quick_plot.png` (sanity line plot by region)


In [17]:
# === Save cleaned outputs ===
clean_path = RESULTS_DIR / "cleaned_rainfall.csv"
clean.to_csv(clean_path, index=False)

summary_txt = clean.describe(include="all").to_string()
(RESULTS_DIR / "summary.txt").write_text(summary_txt, encoding="utf-8")

# Quick plot
plt.figure(figsize=(7,4))
for region, sub in clean.groupby("Region"):
    plt.plot(sub["Date"], sub["Rainfall(mm)"], marker="o", label=region)
plt.title("Rainfall Over Time (cleaned)")
plt.xlabel("Date")
plt.ylabel("Rainfall (mm)")
plt.legend()
plt.tight_layout()
plot_path = RESULTS_DIR / "quick_plot.png"
plt.savefig(plot_path, dpi=160)
plt.close()

print("✅ Cleaned CSV:", clean_path)
print("📝 Summary:", RESULTS_DIR / 'summary.txt')
print("📈 Plot:", plot_path)

✅ Cleaned CSV: D:\DS Projects\pandas-data-cleaning-project\results\cleaned_rainfall.csv
📝 Summary: D:\DS Projects\pandas-data-cleaning-project\results\summary.txt
📈 Plot: D:\DS Projects\pandas-data-cleaning-project\results\quick_plot.png


## 8) What we covered (recap)

- Created a **messy dataset** with common issues (dates, numbers, missing, duplicates, junk)
- **Cleaned** text, dates, and numeric columns using pandas (built on NumPy)
- **Imputed** missing numeric data (median)
- **Deduplicated** rows and saved tidy outputs
- Produced a quick **sanity plot**


